# Nutrient optimization

## todo

* parallel GA with deap
    * DONE: that was too easy!
* API
* set initial seed population with clusters
* GA: remove nutrients that are uncontrained when solving (eg, 1e10)
    * DONE: this doesn't speed up the function
* random seed doesn't work

## Import custom helper functions

* def load_data():
* def do_clust(N,lim,req,nut):
* def evaluate(individual, nut,limt,reqd):
* def makeclusters(nclust,limt,reqd,nutrients ):
* def InitPopulation( pcls, ind_init,nfood, nclust, nseed,clust):
* def generate_ssdum(random, args):

In [1]:
import sys
print(sys.path)
from lib.libraries import *

['/home/pedwards/documents/SpartanSupper/diet_ga', '/home/pedwards/.conda/envs/spartansupper/lib/python37.zip', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/lib-dynload', '', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages/IPython/extensions', '/home/pedwards/.ipython']


## Import external libraries

Most are standard, but we want the glpk solver for cvxopt, which requires the following,
```
$ sudo apt-get install libglpk-dev
$ sudo CVXOPT_BUILD_GLPK=1 pip install cvxopt
```

In [2]:
from os import path

import pickle
import pandas
import numpy
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import normalize
from cvxopt import matrix, solvers # an alternative linear programming library
solvers.options['show_progress'] = False

from sklearn.cluster import KMeans
import random
from time import time
import multiprocessing

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

## Load food & nutrients from database

In [3]:
(nutrients,reqd,limt,food_desc,nutrient_desc)=load_data()
print( '[*] Loaded %d foods from database' % nutrients.shape[0] )

[*] Loaded 4492 foods from database


## Cluster

Observation: the optimization converges faster (and to lower minima) if provided a "seed" population with random baskets of *diverse* foods

Technique: I use a kmeans to find clusters of food types then sample from them (with a multinomial dist)

In [4]:
if not path.exists('clust.pkl'):
    Nclust=15
    clust=makeclusters(Nclust,limt,reqd,nutrients )
    pickle.dump( clust, open( "clust.pkl", "wb" ) )
else:
    clust=pickle.load( open( "clust.pkl", "rb" ) )
    print( '[*] Found pickle file with %d clusters and %d foods' % (clust.max()+1,len(clust)) )
    Nclust=clust.max()+1

[*] Found pickle file with 15 clusters and 4492 foods


## Constants

In [5]:
N_FOODS=6
Nseed=500
NT_DIM=nutrients.shape[0]

## Genetic algorithm

This is the outermost optimization layer (the inner optim is in `evaluate()`)


Todo: play with early stopping. no need to keep going after improvement has ceased or slowed.

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin) # an individual comprises a list (of food IDs)

toolbox = base.Toolbox()
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)
# Attribute generator 
toolbox.register("attr_foodid", random.randrange, NT_DIM)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_foodid, N_FOODS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NT_DIM, indpb=0.1)
#toolbox.register("select", tools.selBest, k=3)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("evaluate", evaluate, nut=nutrients,limt=limt,reqd=reqd)

# used to make a seed population (only) ; per: https://deap.readthedocs.io/en/master/tutorials/basic/part1.html?highlight=seeding#seeding-a-population
toolbox.register("population_guess", InitPopulation, list, creator.Individual, N_FOODS,Nclust,Nseed,limt,reqd,nutrients )

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("min", numpy.min)
stats.register("median", numpy.median)
stats.register("max", numpy.max)

In [7]:
%%time

pop = toolbox.population(n=300) # totally random initial population
#pop = toolbox.population_guess()
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, verbose=True)

gen	nevals	min    	median	max  
0  	300   	1541.21	9e+09 	9e+09
1  	190   	1541.21	9e+09 	9e+09
2  	189   	1198.35	3110.95	9e+09
3  	190   	969.52 	1541.21	9e+09
4  	196   	969.52 	1514.42	9e+09
5  	181   	969.52 	1018.96	9e+09
6  	190   	945.619	969.52 	9e+09
7  	172   	845.362	969.52 	9e+09
8  	172   	845.362	969.52 	9e+09
9  	163   	804.455	945.619	9e+09
10 	184   	804.455	845.362	9e+09
11 	153   	717.242	804.455	9e+09
12 	202   	717.242	804.455	9e+09
13 	184   	697.018	804.379	9e+09
14 	179   	697.018	717.242	9e+09
15 	171   	697.018	717.242	9e+09
16 	181   	697.018	697.018	9e+09
17 	187   	697.018	697.018	9e+09
18 	186   	697.018	697.018	9e+09
19 	164   	697.018	697.018	9e+09
20 	197   	697.018	697.018	9e+09
21 	186   	697.018	697.018	9e+09
22 	175   	697.018	697.018	9e+09
23 	205   	697.018	697.018	9e+09
24 	158   	697.018	697.018	9e+09
25 	162   	697.018	697.018	9e+09
26 	151   	697.018	697.018	9e+09
27 	178   	697.018	697.018	9e+09
28 	179   	697.018	697.018	9e+09
29 	181   	69

In [8]:
best=tools.selBest(pop, k=1)
best=best[0]
evaluate(best, nut=nutrients,limt=limt,reqd=reqd)
nt=nutrients.iloc[best,:]
c = matrix(numpy.repeat(1.0,nt.shape[0]))
np_G= numpy.concatenate(
                        (   nt.transpose().values, 
                            nt.transpose().multiply(-1.0).values,
                            numpy.diag(numpy.repeat(-1,nt.shape[0])) 
                        )
                       ).astype(numpy.double) 
G = matrix( np_G ) 
h = matrix( numpy.concatenate( (
                limt.values, 
                reqd.multiply(-1.0).values, 
                numpy.repeat(0.0,nt.shape[0])
            ) ).astype(numpy.double) )    
o = solvers.lp(c, G, h, solver='glpk')
food_amounts=numpy.array(o['x'])[:,0]


## Print the best "diet"

This is the food and corresponding amount to eat (in grams, sorry bud!). The idea is if you eat all this in a day you have the nutrients you need for the day.

Todo: this would be better if split up into recipes. 

In [9]:
final_foods= [ best[i] for i in range(len(food_amounts)) if food_amounts[i]>1e-6 ] # select those with non-trivial amounts
final_food_amounts= food_amounts[ food_amounts>1e-6 ]

nt=nutrients.iloc[final_foods,:]
df1= nt.join(food_desc).loc[:,['long_desc']] #food_desc.iloc[final_foods]
df2=pandas.DataFrame(final_food_amounts*100,index=df1.index,columns=["amount"])
df3=pandas.DataFrame(nt.loc[:,208].values * df2.loc[:,'amount'].values/100 ,columns=['calories'], index=df2.index)
diet_table=df1.join(df2).join(df3)

In [10]:
diet_table

,long_desc,amount,calories
food_id,,,
42128,"Turkey ham, sliced, extra lean, prepackaged or...",15.569764,19.306507
7925,"Pastrami, beef, 98% fat-free",116.757332,110.919465
16379,"Mothbeans, mature seeds, cooked, boiled, with ...",4.055780,4.745262
14630,"Energy drink, ROCKSTAR, sugar free",706.013055,28.240522
11138,"Cauliflower, frozen, cooked, boiled, drained, ...",1166.250379,221.587572
11095,"Broccoli, frozen, spears, cooked, boiled, drai...",977.543955,273.712307


In [11]:
nutrient_totals=pandas.DataFrame( ( 
                    numpy.dot( numpy.transpose(final_food_amounts), nt.values),
                    reqd,
                    limt
                  ), index=['Total','Amount required','Amount limit'], columns=nt.columns).transpose()

In [12]:
A=nt.join(df1).set_index('long_desc').transpose() * final_food_amounts
nutrient_report=A.join(nutrient_desc).join(nutrient_totals).set_index('name')
nutrient_report

,"Turkey ham, sliced, extra lean, prepackaged or deli-sliced","Pastrami, beef, 98% fat-free","Mothbeans, mature seeds, cooked, boiled, with salt","Energy drink, ROCKSTAR, sugar free","Cauliflower, frozen, cooked, boiled, drained, without salt","Broccoli, frozen, spears, cooked, boiled, drained, without salt",Total,Amount required,Amount limit
name,,,,,,,,,
Protein,3.051674,22.884438,0.316756,1.765033,18.776632,30.303862,77.098393,53.459999,1.000000e+10
Total lipid (fat),0.591651,1.354385,0.022307,0.564810,2.565751,1.075298,6.174202,0.000000,1.000000e+10
"Carbohydrate, by difference",0.456194,1.798063,0.850091,4.942091,43.734390,52.396355,104.177186,0.000000,1.000000e+10
Ash,0.260015,3.969749,0.058809,4.377281,4.898252,6.940562,20.504668,0.000000,1.000000e+10
Energy,19.306505,110.919464,4.745262,28.240522,221.587585,273.712311,658.511636,0.000000,1.000000e+10
Starch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Sucrose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Glucose (dextrose),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Fructose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
